In [1]:
from mrequests import MattermostRequests

In [2]:
# Mattermost credentials should to be placed outside of this repo.
mr = MattermostRequests('../mattermost_credentials.json', 'mattermost-openapi-v4.yaml')

#### API testing

In [ ]:
team_id = mr.get_teams()[0]['id']
channel_id = mr.get_public_channels(team_id=team_id)[0]['id']

In [ ]:
mr.create_a_post(channel_id=channel_id, message='this is a new message')

A motor feladata, hogy végrehajt egy teszt sort, és értékeli az eredményt. Addig folyik a tesztelés, amíg egy hibás válasz nem érkezik. (Hiba az is, ha nem érkezik válasz egy megadott időn belül, de itt figyelembe kell venni a hálózati késleltetést is.) A fuzzingot ezen a ponton úgy kéne megvalósítani, hogy az előbbi fuzzoló metódust használva a teszt alatt lévő API metódus össze lehetséges paraméterét végigtesztelni az összes lehetséges kombinációban. (Ez elvben nagyon sok, úgyhogy inkább az összes lehetséges teszt egy random részhalmazával érdemes próbálkozni.) A visszajövő válaszokat pedig minden esetben a séma alapján validálni kell (lásd 2. feladat).

In [ ]:
def run_test(flow, input_data):
    for action, response_processor in flow:
        result = action(action)
        evaluate_result(result)
        input_data = response_processor(result)
        

In [3]:
mr.get_methods()

[('activate_plugin', ['plugin_id']),
 ('add_log_message', ['body']),
 ('add_multiple_users_to_team', ['team_id', 'body']),
 ('add_user_to_channel', ['channel_id', 'body']),
 ('add_user_to_team', ['team_id', 'body']),
 ('add_user_to_team_from_invite', ['hash', 'data', 'invite_id']),
 ('attach_mobile_device', ['body']),
 ('autocomplete_channels', ['team_id', 'name']),
 ('autocomplete_custom_emoji', ['name']),
 ('autocomplete_users', ['team_id', 'channel_id', 'name']),
 ('cancel_a_job', ['job_id']),
 ('check_if_team_exists', ['name']),
 ('check_mfa', ['body']),
 ('check_system_health', []),
 ('create_a_channel', ['body']),
 ('create_a_command', ['body']),
 ('create_a_custom_emoji', ['image', 'emoji']),
 ('create_a_direct_message_channel', ['body']),
 ('create_a_group_message_channel', ['body']),
 ('create_a_new_job', ['body']),
 ('create_a_post', ['post']),
 ('create_a_team', ['body']),
 ('create_a_user', ['body']),
 ('create_a_user_access_token', ['user_id', 'token']),
 ('create_an_incom